In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os 
import yaml
from dotenv import load_dotenv
load_dotenv('../../../.env')

model=os.environ.get('model')
print(model)

azure/gpt-4


In [4]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/1_project_planning/agents.yaml',
    'tasks': 'config/1_project_planning/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Create Pydantic Models for Structured Output

In [5]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

In [6]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

In [7]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



In [8]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner\n
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build t

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [9]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0012


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,8037,4709,0,3328,3


## Result

In [10]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Gather requirements and create project documentation',
   'estimated_time_hours': 20.0,
   'required_resources': ['John Doe']},
  {'task_name': 'Create wireframes for responsive design',
   'estimated_time_hours': 48.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Develop visual language for the website',
   'estimated_time_hours': 40.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Create high-fidelity mockups',
   'estimated_time_hours': 68.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Set up development environment',
   'estimated_time_hours': 12.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Develop front-end interface',
   'estimated_time_hours': 144.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Implement navigation system',
   'estimated_time_hours': 32.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Build content management system for blog',
   'estimated_time_hours': 52.0,
 

## Inspect further

In [11]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Gather requirements and create project documentation,20.000000,['John Doe']
1,Create wireframes for responsive design,48.000000,['Bob Smith']
2,Develop visual language for the website,40.000000,['Bob Smith']
3,Create high-fidelity mockups,68.000000,['Bob Smith']
4,Set up development environment,12.000000,['Jane Doe']
5,Develop front-end interface,144.000000,['Jane Doe']
6,Implement navigation system,32.000000,['Jane Doe']
7,Build content management system for blog,52.000000,['Jane Doe']
8,Implement SEO best practices,28.000000,['Jane Doe']
9,Optimize website for speed,20.000000,['Jane Doe']


### Inspecting Milestones

In [12]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Project Documentation Complete,['Gather requirements and create project documentation']
1,Design Phase Complete,"['Create wireframes for responsive design', 'Develop visual language for the website', 'Create high-fidelity mockups']"
2,Development Environment Set Up,['Set up development environment']
3,Front-end Interface Complete,['Develop front-end interface']
4,Navigation and CMS Complete,"['Implement navigation system', 'Build content management system for blog']"
5,SEO and Performance Optimization Complete,"['Implement SEO best practices', 'Optimize website for speed']"
6,Social Media Integration and Testimonials Complete,"['Integrate social media functionality', 'Implement a dynamic testimonials section']"
7,Quality Assurance Complete,"['Conduct thorough QA Testing', 'Address and fix reported issues']"
8,Final Client Review,['Final client review and feedback']
9,Website Deployment,['Implement client feedback and prepare for launch']
